 # Importing scientific output, if DOIs are provided

This is the code about fetching publication information of researchers, if digital object information (DOIs) is avialable.

There is a need for automation of the publication information of the researchers. Once this
information is available, the information can be automatically linked to the relevant researcher. Furthermore, this information can be used for a visualization for finding conflicts of interest with other researchers.

A researcher's publication information will be processed via a free (cross-ref) API, provided DOIs of the publications are available. For proof of concept I have DOIs of my own four publications collected in an input excel file. The json format of Cross-ref API has been edited with python scripts. Excel output file is made of list of authors and number of citations.

Shortcomings The format of output file is not yet as it should be. Two wishes that haven't been fulfilled yet
to be:
ˆ columns 'family name' and 'given name' side by side and
ˆ split year-month column

In [ ]:
#Importing various modules required for analysis
#You excel should have a single column of DOI's. The header of the column should be 'DOIs'
import os
os.getcwd()
import tkinter as tk
from tkinter import filedialog
import pandas as pd
#Importing the dois with Tkinter module
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue')
canvas1.pack()

def getExcel ():
    global df
    
    import_file_path = filedialog.askopenfilename()
    df = pd.read_excel (import_file_path)
    #print (df)
    
browseButton_Excel = tk.Button(text='Import Excel File', command=getExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=browseButton_Excel)

root.mainloop()

In [ ]:
# If viewing in jupyter notebook or running first time, then uncomment the next line and run the cell.
#!pip install crossrefapi

In [ ]:
#conversion of excel of dois to list format. 
doi=df['DOIs'].to_list()

In [ ]:
#Let's start by looking briefly at"works". The route refers to items identified by a DOI in the index. These can be articles, books, components, etc.
from crossref.restful import Works
works = Works()

In [ ]:
#extraction of author information
def get_author_info():
    authors_info=record['author']
    for authors in authors_info:
        familyname=authors['family']
        givenname=authors['given']
        familynames.append(familyname)
        givennames.append(givenname)

In [ ]:
#extraction of family names and given names
familynamess=[]
givennamess=[]
for i in doi:
    record=works.doi(i)
    familynames=[]
    givennames=[]
    get_author_info()
    familynamess.append(familynames)
    givennamess.append(givennames)
df_famm=pd.DataFrame(familynamess)
df_given=pd.DataFrame(givennamess)


In [ ]:
#joining family and given names dataframes
df_author=pd.concat([df_famm,df_given], axis=1)

In [ ]:
#list of authors, and other description
title=[]
journal=[]

volume=[]
issue=[]
page=[]
cit=[]
reference=[]
# Using for loop 
for i in doi: 
    record = works.doi(i)
    title.append(record['title'])
    journal.append(record['container-title'])
    volume.append(record['volume'])
    issue.append(record['member'])
    page.append(record['page'])
    cit.append(record['is-referenced-by-count'])
    reference.append(record['references-count'])
    

In [ ]:
#extraction of publication dates
pub_date=[]
for i in doi: 
    record = works.doi(i)
    newrow=record['published-print']
    pub_date.append(newrow)
df_date= pd.DataFrame(pub_date)
df_date.rename(columns={'date-parts': 'Date_list'}, inplace=True)

In [ ]:
Date_list=df_date['Date_list'].tolist()

In [ ]:
Date_list=[str(item) for item in Date_list]

In [ ]:
df_date['year-month']=Date_list

In [ ]:
#delete the dataframe because it it not needed anymore
del df_date['Date_list']

In [ ]:
#writing various dataframes for the information
df_title=pd.DataFrame(title, columns=['title'])
df_journal=pd.DataFrame(journal, columns=['journal'])
df_volume=pd.DataFrame(volume, columns=['volume'])
df_issue=pd.DataFrame(issue, columns=['issue'])
df_page=pd.DataFrame(page, columns=['pages'])
df_cit=pd.DataFrame(cit, columns=['citation'])
df_ref=pd.DataFrame(reference, columns=['references'])

In [ ]:
#joining all the dataframes with the information
df_all=pd.concat([df, df_title, df_author, df_journal, df_date, df_volume,df_issue, df_page, df_cit,df_ref], axis=1)

In [ ]:
#Printing the output
#The openpyxl.utils.dataframe.dataframe_to_rows() function provides a simple way to work with Pandas Dataframes:

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook


wb = Workbook()
ws = wb.active
#ws = wb.create_sheet("Samenvatting", 0) 
#ws.title='Trial'

for r in dataframe_to_rows(df_all, index=False, header=True):
    ws.append(r)

    #saving the excel workbook
Exportfile_name=input('Give export file name ',)# write the name of the file with extension .xlsx
wb.save(Exportfile_name)